<a href="https://colab.research.google.com/github/Prof-it/geo-spatial-Berlin-EVC-placement/blob/main/geo_spatial_Berlin_EVC_placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
pip install pymoo pandas geopandas requests

In [33]:
#!/usr/bin/env python

"""
SCRIPT 1: NSGA-II GENERATOR (Run this on your local computer)

This script generates the NSGA-II optimization results for the
Berlin EV charger placement problem.

It MUST be run locally after installing 'pymoo'.
"""

import os
import json
import math
import random
import glob
import sys
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
import geopandas as gpd
import requests

# --- Prerequisite: pymoo library ---
try:
    from pymoo.core.problem import Problem
    from pymoo.algorithms.moo.nsga2 import NSGA2
    from pymoo.operators.sampling.rnd import IntegerRandomSampling
    from pymoo.operators.crossover.sbx import SBX
    from pymoo.operators.mutation.pm import PM
    from pymoo.operators.repair.rounding import RoundingRepair
    from pymoo.optimize import minimize
except ImportError:
    print("="*50)
    print("CRITICAL ERROR: 'pymoo' library not found.")
    print("Please install it on your local machine: pip install pymoo pandas geopandas requests")
    print("This script cannot run without it.")
    print("="*50)
    sys.exit(1) # Force script to stop

# --- 1. Baseline Data Functions ---

OUTPUT_DIR = Path("./simulation_outputs_k100")
DISTRICTS_GEOJSON_PATH = "berlin_districts.geojson"
DISTRICTS_DOWNLOAD_URL = "https://raw.githubusercontent.com/funkeinteraktiv/Berlin-Geodaten/master/berlin_bezirke.geojson"

POP_BY_DISTRICT = {
    "Mitte": 397134, "Friedrichshain-Kreuzberg": 293454, "Pankow": 424307,
    "Charlottenburg-Wilmersdorf": 343081, "Spandau": 257091,
    "Steglitz-Zehlendorf": 310446, "Tempelhof-Schöneberg": 355868,
    "Neukölln": 330017, "Treptow-Köpenick": 294081,
    "Marzahn-Hellersdorf": 291948, "Lichtenberg": 311881,
    "Reinickendorf": 268792
}
N_DISTRICTS = len(POP_BY_DISTRICT)

def find_latest_bnetza_file():
    files = glob.glob("Ladesaeulenregister_BNetzA_*.csv")
    if not files: return None
    files.sort()
    print(f"Using data file: {files[-1]}")
    return files[-1]

def download_berlin_districts():
    if os.path.exists(DISTRICTS_GEOJSON_PATH):
        print(f"Using existing district file: {DISTRICTS_GEOJSON_PATH}")
        return True
    print(f"Downloading Berlin district boundaries from {DISTRICTS_DOWNLOAD_URL}...")
    try:
        response = requests.get(DISTRICTS_DOWNLOAD_URL, timeout=30)
        response.raise_for_status()
        with open(DISTRICTS_GEOJSON_PATH, 'w', encoding='utf-8') as f:
            f.write(response.text)
        print(f"✅ District boundaries saved as {DISTRICTS_GEOJSON_PATH}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"District download failed: {e}")
        return False

def find_col(df_columns, possibilities):
    """Finds the correct column name from a list of possibilities."""
    for col in possibilities:
        if col in df_columns:
            return col
    # If not found, try lowercase versions robustly
    possibilities_lower = {p.lower() for p in possibilities}
    for col in df_columns:
        if col.lower() in possibilities_lower:
            return col
    return None


def prepare_baseline_data_from_bneza(bnetza_file_path):
    try:
        gdf_districts = gpd.read_file(DISTRICTS_GEOJSON_PATH).to_crs(epsg=4326)
    except Exception as e:
        print(f"❌ CRITICAL: Could not read {DISTRICTS_GEOJSON_PATH}. Error: {e}")
        return None

    DISTRICT_COL = 'name' # Correct column in the GeoJSON
    if DISTRICT_COL not in gdf_districts.columns:
        print(f"❌ CRITICAL: Could not find district name column '{DISTRICT_COL}' in {DISTRICTS_GEOJSON_PATH}.")
        print(f"Found columns: {gdf_districts.columns.to_list()}")
        return None

    try:
        # *** FINAL FIX: Changed header=10 to header=11 ***
        df_bnetza = pd.read_csv(
            bnetza_file_path,
            sep=';',
            encoding='latin-1',
            decimal=',',
            header=11,  # Skip first 11 rows, read headers from row 12 (0-indexed)
            low_memory=False
        )
    except Exception as e:
        print(f"❌ CRITICAL: Failed to read BNetzA CSV file (using header=11, latin-1). Error: {e}")
        return None

    # --- Robust Column Finding ---
    all_cols = df_bnetza.columns
    # Clean up column names (remove leading/trailing spaces if any)
    df_bnetza.columns = df_bnetza.columns.str.strip()
    all_cols = df_bnetza.columns # Update after stripping

    city_col = find_col(all_cols, ['Ort'])
    date_col = find_col(all_cols, ['Inbetriebnahmedatum'])
    lat_col = find_col(all_cols, ['Breitengrad'])
    lon_col = find_col(all_cols, ['Längengrad'])
    count_col = find_col(all_cols, ['Anzahl Ladepunkte'])

    if not all([city_col, date_col, lat_col, lon_col, count_col]):
        print("❌ CRITICAL: Could not find one or more required columns AFTER skipping header rows (header=11).")
        if not city_col: print("   - Missing: 'Ort' (City)")
        if not date_col: print("   - Missing: 'Inbetriebnahmedatum' (Date)")
        if not lat_col: print("   - Missing: 'Breitengrad' (Latitude)")
        if not lon_col: print("   - Missing: 'Längengrad' (Longitude)")
        if not count_col: print("   - Missing: 'Anzahl Ladepunkte' (Count)")
        print(f"Found columns in row 12: {all_cols.to_list()}") # Show columns found after skipping
        return None

    print(f"Using columns: City='{city_col}', Date='{date_col}', Lat='{lat_col}', Lon='{lon_col}', Count='{count_col}'")

    df_berlin = df_bnetza[df_bnetza[city_col] == 'Berlin'].copy()

    df_berlin[date_col] = pd.to_datetime(
        df_berlin[date_col], dayfirst=True, errors='coerce'
    )
    df_berlin = df_berlin.dropna(subset=[date_col])
    cutoff = datetime(2024, 12, 31)
    df_berlin_filtered = df_berlin[df_berlin[date_col] <= cutoff].copy()

    df_berlin_filtered[lat_col] = pd.to_numeric(df_berlin_filtered[lat_col], errors='coerce')
    df_berlin_filtered[lon_col] = pd.to_numeric(df_berlin_filtered[lon_col], errors='coerce')
    df_berlin_filtered = df_berlin_filtered.dropna(subset=[lat_col, lon_col])

    gdf_points = gpd.GeoDataFrame(
        df_berlin_filtered,
        geometry=gpd.points_from_xy(df_berlin_filtered[lon_col], df_berlin_filtered[lat_col]),
        crs="EPSG:4326"
    )

    gdf_joined = gpd.sjoin(gdf_points, gdf_districts, how='left', predicate='within')

    # Group by the correct district and count columns
    canon_counts = gdf_joined.groupby(DISTRICT_COL)[count_col].sum().to_dict()

    baseline = []
    for b_name in POP_BY_DISTRICT.keys():
        baseline.append({
            "district": b_name,
            "existing_chargers": int(canon_counts.get(b_name, 0)),
            "population": POP_BY_DISTRICT[b_name],
        })
    df_base = pd.DataFrame(baseline).set_index("district")
    total_pop = df_base["population"].sum()
    df_base["bev_est"] = (50802 * df_base["population"] / total_pop).round().astype(int)
    df_base["bev_per_charger_initial"] = df_base["bev_est"] / df_base["existing_chargers"].replace(0, np.nan)
    df_base["bev_per_charger_initial"] = df_base["bev_per_charger_initial"].fillna(df_base["bev_est"])

    print("✅ Baseline data prepared successfully.")
    return df_base

# --- 2. Simulation Functions ---

def gini(array):
    arr = np.array(array, dtype=float)
    if arr.size == 0 or np.all(arr == 0): return 0.0
    arr = arr.flatten()
    if np.any(arr < 0): arr = arr - arr.min()
    n = arr.size
    sorted_arr = np.sort(arr)
    cumvals = np.cumsum(sorted_arr, dtype=float)
    if cumvals[-1] == 0: return 0.0
    g = (n + 1 - 2 * np.sum(cumvals) / cumvals[-1]) / n
    return g

def run_queue_simulation(df_alloc, freq_per_bev_week=0.2, mean_service_minutes=45, seed=42):
    random.seed(seed)
    np.random.seed(seed)
    period_hours = 7 * 24
    results = {}
    for idx, row in df_alloc.iterrows():
        bev = int(row["bev_est"])
        servers = int(row["post_chargers"])
        if servers <= 0:
            results[idx] = {"avg_wait_min": None}
            continue

        lam = bev * freq_per_bev_week
        n_arrivals = np.random.poisson(lam)
        arrival_times = np.sort(np.random.uniform(0, period_hours, size=n_arrivals))
        service_times = np.random.exponential(scale=(mean_service_minutes / 60.0), size=n_arrivals)

        server_free = [0.0] * servers
        waits = []
        for at, st in zip(arrival_times, service_times):
            idx_server = min(range(servers), key=lambda i: server_free[i])
            free_time = server_free[idx_server]
            wait = max(0.0, free_time - at)
            start = at + wait
            finish = start + st
            server_free[idx_server] = finish
            waits.append(wait * 60.0)

        avg_wait = float(np.mean(waits)) if waits else 0.0
        results[idx] = {"avg_wait_min": avg_wait}
    return results

# --- 3. NSGA-II Optimization Problem Definition ---

class ChargerAllocationProblem(Problem):
    """
    Defines the multi-objective optimization problem for pymoo.
    """
    def __init__(self, df_base, K):
        self.df_base = df_base
        self.K = K

        super().__init__(
            n_var=N_DISTRICTS,
            n_obj=2,
            n_constr=1,
            xl=0,
            xu=K,
            elementwise=True,
            vtype=int
        )

    def _evaluate(self, x, out, *args, **kwargs):
        # Constraint: sum of allocated chargers must equal K
        g1 = (np.sum(x) - self.K) ** 2
        out["G"] = [g1]

        # --- Objectives ---
        df_alloc = self.df_base.copy()
        df_alloc["new_alloc"] = x
        df_alloc["post_chargers"] = df_alloc["existing_chargers"] + df_alloc["new_alloc"]

        df_alloc["post_bev_per_charger"] = df_alloc["bev_est"] / df_alloc["post_chargers"].replace(0, np.nan)
        df_alloc["post_bev_per_charger"] = df_alloc["post_bev_per_charger"].fillna(df_alloc["bev_est"])

        # Objective 1: Minimize Gini
        f1 = gini(df_alloc["post_bev_per_charger"].values)

        # Objective 2: Minimize Average Wait Time
        sim_res = run_queue_simulation(df_alloc)
        all_waits = [v["avg_wait_min"] for v in sim_res.values() if v["avg_wait_min"] is not None]

        if not all_waits:
             f2 = 1e6 # Assign a large penalty if simulation fails
        else:
             f2 = np.mean(all_waits)

        out["F"] = [f1, f2]

# --- 4. Main Execution ---

def main():
    OUTPUT_DIR.mkdir(exist_ok=True)

    # 1. Load baseline data
    bnetza_file = find_latest_bnetza_file()
    if not bnetza_file:
        print("❌ CRITICAL: No BNetzA file found. Exiting.")
        return
    if not download_berlin_districts():
        print("❌ CRITICAL: Could not download district boundaries. Exiting.")
        return

    df_base = prepare_baseline_data_from_bneza(bnetza_file)
    if df_base is None:
        print("❌ CRITICAL: Data preparation failed. Exiting.")
        return

    # 2. Define K values to run
    K_values = [25, 50, 100, 150]
    all_results = []

    for K in K_values:
        print(f"\n--- Running NSGA-II Optimization for K = {K} ---")

        problem = ChargerAllocationProblem(df_base, K)

        algorithm = NSGA2(
            pop_size=100,
            sampling=IntegerRandomSampling(),
            crossover=SBX(prob=0.9, eta=15, vtype=int, repair=RoundingRepair()),
            mutation=PM(prob=0.1, eta=20, vtype=int, repair=RoundingRepair()),
            eliminate_duplicates=True
        )

        res = minimize(
            problem,
            algorithm,
            ('n_gen', 200),
            verbose=True,
            seed=42
        )

        print(f"--- Optimization for K = {K} Complete ---")

        if res.X is not None:
            for i in range(len(res.X)):
                solution_vars = res.X[i]
                solution_objs = res.F[i]

                # We store Avg_Wait_Min in the 'Weighted_Coverage_Score' column
                # to match the format of the file you provided.
                all_results.append({
                    "Replication": 0,
                    "K": K,
                    "Gini_post": solution_objs[0],
                    "Weighted_Coverage_Score": solution_objs[1],
                    "Allocation": str(list(solution_vars))
                })
        else:
            print(f"No valid solutions found for K = {K}.")

    # 7. Save the final DataFrame to CSV
    df_results = pd.DataFrame(all_results)

    output_filename = "nsga2_all_k_results_GENERATED.csv"
    df_results.to_csv(output_filename, index=False)

    print("\n=======================================================")
    print(f"✅ All optimizations complete!")
    print(f"Results saved to: {output_filename}")
    print("Column 'Weighted_Coverage_Score' contains 'Avg_wait_min'.")
    print("=======================================================")

if __name__ == "__main__":
    main()

Using data file: Ladesaeulenregister_BNetzA_2025-07-18.csv
Using existing district file: berlin_districts.geojson
❌ CRITICAL: Could not find one or more required columns AFTER skipping header rows (header=11).
   - Missing: 'Ort' (City)
   - Missing: 'Inbetriebnahmedatum' (Date)
   - Missing: 'Breitengrad' (Latitude)
   - Missing: 'Längengrad' (Longitude)
   - Missing: 'Anzahl Ladepunkte' (Count)
Found columns in row 12: ['1010338', 'Albwerk Elektro- und Kommunikationstechnik GmbH', 'Albwerk Elektro- und Kommunikationstechnik GmbH.1', 'In Betrieb', 'Normalladeeinrichtung', '2', '22', '11.01.2020', 'Ennabeurer Weg', '0', 'Unnamed: 10', '72535', 'Heroldstatt', 'Landkreis Alb-Donau-Kreis', 'Baden-Württemberg', '48,442398', '9,659075', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Keine Angabe', 'Unnamed: 21', 'Unnamed: 22', 'AC Typ 2 Steckdose', '22.1', 'Unnamed: 25', 'Unnamed: 26', 'AC Typ 2 Steckdose.1', '22.2', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 

In [36]:
#!/usr/bin/env python

"""
SCRIPT 2: CHART GENERATOR (Using chargers_locations.geojson)

This script collects data from chargers_locations.geojson, runs simple
simulations (A, B, C), and generates all charts.

It REQUIRES the files 'chargers_locations.geojson' and
'nsga2_all_k_results.csv' to be present.
"""

import os
import json
import math
import random
import glob
import sys
from pathlib import Path
from datetime import datetime, timezone # Import timezone for comparison

# Core data handling and analysis
import pandas as pd
import numpy as np
import geopandas as gpd
import requests

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from IPython.display import display

# --- 0. Global Setup ---
OUTPUT_DIR = Path("./simulation_outputs_final_geojson")
NSGA2_FILE_PATH = "nsga2_all_k_results.csv"
CHARGERS_GEOJSON_PATH = "chargers_locations.geojson" # Use this file

# Population data remains the same
POP_BY_DISTRICT = {
    "Mitte": 397134, "Friedrichshain-Kreuzberg": 293454, "Pankow": 424307,
    "Charlottenburg-Wilmersdorf": 343081, "Spandau": 257091,
    "Steglitz-Zehlendorf": 310446, "Tempelhof-Schöneberg": 355868,
    "Neukölln": 330017, "Treptow-Köpenick": 294081,
    "Marzahn-Hellersdorf": 291948, "Lichtenberg": 311881,
    "Reinickendorf": 268792
}
N_DISTRICTS = len(POP_BY_DISTRICT)

# --- 1. Helper Functions ---

def canonical_name(name):
    """Maps variant district names from GeoJSON properties to a canonical name."""
    key = str(name).strip()
    # Map known variants observed in the GeoJSON properties
    if "Friedrichshain" in key and "Kreuzberg" in key: return "Friedrichshain-Kreuzberg"
    if "Friedrichshain-Kr" in key: return "Friedrichshain-Kreuzberg"
    if "Mitte" in key: return "Mitte"
    if "Pankow" in key: return "Pankow"
    if "Treptow" in key or "Köpenick" in key: return "Treptow-Köpenick"
    if "Neuk" in key or "Neukölln" in key: return "Neukölln"
    if "Charlottenburg" in key or "Wilmersdorf" in key: return "Charlottenburg-Wilmersdorf"
    if "Spandau" in key: return "Spandau"
    if "Steglitz" in key or "Zehlendorf" in key: return "Steglitz-Zehlendorf"
    if "Tempelhof" in key or "Schöneberg" in key or "Schoeneberg" in key: return "Tempelhof-Schöneberg"
    if "Marzahn" in key or "Hellersdorf" in key: return "Marzahn-Hellersdorf"
    if "Lichtenberg" in key: return "Lichtenberg"
    if "Reinickendorf" in key: return "Reinickendorf"
    # Fallback if no specific match
    return key

def prepare_baseline_data_from_geojson(geojson_file_path):
    """
    Loads GeoJSON data, processes properties, and returns the aggregated
    baseline DataFrame and coordinate dictionary.
    """
    print(f"Preparing baseline data from {geojson_file_path}...")

    # 1. Load GeoJSON
    try:
        gdf = gpd.read_file(geojson_file_path)
        if gdf.crs is None:
            gdf.set_crs(epsg=4326, inplace=True)
        elif gdf.crs.to_epsg() != 4326:
            gdf = gdf.to_crs(epsg=4326)

    except Exception as e:
        print(f"❌ CRITICAL: Could not read {geojson_file_path}. Error: {e}")
        return None, None

    # 2. Filter by date
    # Make cutoff timezone-aware (UTC) as GeoJSON dates likely have 'Z'
    cutoff = datetime(2024, 12, 31, 23, 59, 59, tzinfo=timezone.utc)

    # Handle timezone info if present (e.g., 'Z' suffix)
    gdf['inbetriebnahme_dt'] = pd.to_datetime(gdf['inbetriebnahme'], errors='coerce', utc=True)

    # Filter rows where conversion was successful and date is <= cutoff
    gdf_filtered = gdf[gdf['inbetriebnahme_dt'].notna() & (gdf['inbetriebnahme_dt'] <= cutoff)].copy()

    print(f"Found {len(gdf_filtered)} chargers commissioned by {cutoff.date()}.")

    # 3. Aggregate counts and coordinates by canonical district name
    canon_counts = {}
    canon_coords = {}

    for idx, row in gdf_filtered.iterrows():
        # Try multiple potential district name columns
        bezirk_raw = row.get('bezirk') or row.get('bezirk_name') or row.get('district') or "Unknown"
        canon_dist = canonical_name(bezirk_raw)

        if canon_dist not in POP_BY_DISTRICT:
            continue # Skip chargers we can't map

        # Use 'anzahl_ladepunkte', default to 1
        num_chargers = pd.to_numeric(row.get('anzahl_ladepunkte'), errors='coerce')
        if pd.isna(num_chargers) or num_chargers < 1:
            num_chargers = 1
        num_chargers = int(num_chargers)

        canon_counts[canon_dist] = canon_counts.get(canon_dist, 0) + num_chargers

        # Extract coordinates
        geom = row.geometry
        if geom is None or geom.is_empty:
            continue

        coords = None
        if geom.geom_type == 'Point':
            coords = (geom.x, geom.y) # lon, lat
        elif geom.geom_type == 'MultiPoint':
            if len(geom.geoms) > 0:
                 coords = (geom.geoms[0].x, geom.geoms[0].y)

        if coords:
            canon_coords.setdefault(canon_dist, [])
            for _ in range(num_chargers):
                canon_coords[canon_dist].append(coords)

    # 4. Build final baseline DataFrame
    baseline = []
    berlin_center = (13.4050, 52.5200) # lon, lat fallback

    for b_name in POP_BY_DISTRICT.keys():
        coords_list = canon_coords.get(b_name, [])
        centroid_x = np.mean([c[0] for c in coords_list]) if coords_list else berlin_center[0]
        centroid_y = np.mean([c[1] for c in coords_list]) if coords_list else berlin_center[1]

        baseline.append({
            "district": b_name,
            "existing_chargers": int(canon_counts.get(b_name, 0)),
            "population": POP_BY_DISTRICT[b_name],
            "centroid_x": centroid_x,
            "centroid_y": centroid_y
        })

    df_base = pd.DataFrame(baseline).set_index("district")

    total_pop = df_base["population"].sum()
    df_base["bev_est"] = (50802 * df_base["population"] / total_pop).round().astype(int)

    df_base["bev_per_charger_initial"] = df_base["bev_est"] / df_base["existing_chargers"].replace(0, np.nan)
    df_base["bev_per_charger_initial"] = df_base["bev_per_charger_initial"].fillna(df_base["bev_est"])

    print("✅ Baseline data prepared successfully from GeoJSON.")
    # Add check for charger counts
    if df_base["existing_chargers"].sum() == 0:
        print("⚠️ WARNING: No existing chargers found after filtering. Results may be inaccurate.")
    else:
        print(f"Total existing chargers found: {df_base['existing_chargers'].sum()}")
    # print(df_base.head()) # Optional: Display head for verification
    return df_base, canon_coords


def gini(array):
    """Calculates the Gini coefficient."""
    arr = np.array(array, dtype=float)
    if arr.size == 0 or np.all(arr == 0): return 0.0
    arr = arr.flatten()
    if np.any(arr < 0): arr = arr - arr.min()
    n = arr.size
    sorted_arr = np.sort(arr)
    cumvals = np.cumsum(sorted_arr, dtype=float)
    if cumvals[-1] == 0: return 0.0
    g = (n + 1 - 2 * np.sum(cumvals) / cumvals[-1]) / n
    return g

def allocate_chargers(df, K, strategy):
    """Applies one of three allocation strategies (A, B, C)."""
    df2 = df.copy()
    df2["new_alloc"] = 0

    if K == 0: # Handle case where K is 0
        df2["post_chargers"] = df2["existing_chargers"]
        df2["post_bev_per_charger"] = df2["bev_per_charger_initial"] # Use initial ratio
        df2["gini_post"] = gini(df2["post_bev_per_charger"].values)
        return df2

    if strategy == "A": # Max Equity
        order = df2.sort_values("bev_per_charger_initial", ascending=False).index.tolist()
    elif strategy == "C": # Max Utility
        order = df2.sort_values("population", ascending=False).index.tolist()
    else: # Strategy B (Balanced)
        base = K // len(df2)
        rem = K % len(df2)
        df2["new_alloc"] = base
        order = df2.sort_values("bev_per_charger_initial", ascending=False).index.tolist()
        for i in range(rem):
            df2.at[order[i % len(order)], "new_alloc"] += 1
        K = 0 # Allocation done

    # Allocate remaining for A and C
    if strategy in ["A", "C"] and K > 0:
        for i in range(K):
            if not order: # Safety check if order list is somehow empty
                break
            target = order[i % len(order)]
            df2.at[target, "new_alloc"] += 1

    df2["post_chargers"] = df2["existing_chargers"] + df2["new_alloc"]
    df2["post_bev_per_charger"] = df2["bev_est"] / df2["post_chargers"].replace(0, np.nan)
    df2["post_bev_per_charger"] = df2["post_bev_per_charger"].fillna(df2["bev_est"])
    df2["gini_post"] = gini(df2["post_bev_per_charger"].values)
    return df2

def run_queue_simulation(df_alloc, freq_per_bev_week=0.2, mean_service_minutes=45, seed=42):
    """Runs a simple M/M/c queuing simulation per district."""
    random.seed(seed)
    np.random.seed(seed)
    period_hours = 7 * 24
    results = {}
    for idx, row in df_alloc.iterrows():
        bev = int(row["bev_est"])
        servers = int(row["post_chargers"])
        if servers <= 0:
            results[idx] = {"avg_wait_min": None, "pct_waited": None, "utilization": 0.0, "n_arrivals": 0}
            continue

        lam = bev * freq_per_bev_week
        n_arrivals = np.random.poisson(lam)
        arrival_times = np.sort(np.random.uniform(0, period_hours, size=n_arrivals))
        service_times = np.random.exponential(scale=(mean_service_minutes / 60.0), size=n_arrivals)

        server_free = [0.0] * servers
        waits = []
        total_service_time = 0.0

        for at, st in zip(arrival_times, service_times):
            idx_server = min(range(servers), key=lambda i: server_free[i])
            free_time = server_free[idx_server]
            wait = max(0.0, free_time - at)
            start = at + wait
            finish = start + st
            server_free[idx_server] = finish
            waits.append(wait * 60.0)
            total_service_time += st

        avg_wait = float(np.mean(waits)) if waits else 0.0
        pct_waited = float(sum(1 for w in waits if w > 1e-6) / len(waits)) if waits else 0.0
        utilization = total_service_time / (servers * period_hours) if servers * period_hours > 0 else 0.0
        results[idx] = {"avg_wait_min": avg_wait, "pct_waited": pct_waited, "utilization": utilization, "n_arrivals": n_arrivals}
    return results

# --- 2. Main Analysis Functions ---

def run_analysis_1_district_sim(df_base):
    """
    Runs the district-level simulation for K=100, 150, 500, 1000.
    """
    print("\n--- Running Analysis 1: District-Level Simulation (A, B, C) ---")
    Ks = [100, 150, 500, 1000]
    strategies = ["A", "B", "C"]
    summary_rows = []
    all_allocations = {}

    for K in Ks:
        for s in strategies:
            df_alloc = allocate_chargers(df_base, K, s)
            sim_res = run_queue_simulation(df_alloc, freq_per_bev_week=0.2, mean_service_minutes=45)

            # Use np.nanmean to handle districts where simulation might not yield results
            valid_waits = [v["avg_wait_min"] for v in sim_res.values() if v["avg_wait_min"] is not None]
            avg_wait = np.mean(valid_waits) if valid_waits else np.nan # Use np.mean

            valid_pct_waited = [v["pct_waited"] for v in sim_res.values() if v["pct_waited"] is not None]
            pct_waited = np.mean(valid_pct_waited) if valid_pct_waited else np.nan # Use np.mean

            valid_util = [v["utilization"] for v in sim_res.values()]
            util = np.mean(valid_util) if valid_util else np.nan # Use np.mean


            pop_served = df_alloc.loc[df_alloc["post_chargers"] > 0, "population"].sum()
            coverage_pct = 100 * pop_served / df_alloc["population"].sum()
            gini_post = df_alloc["gini_post"].iloc[0]

            summary_rows.append({
                "K": K, "Strategy": s, "Gini_post": gini_post, "Coverage_pct": coverage_pct,
                "Avg_wait_min": avg_wait, "Pct_arrivals_waited": pct_waited, "Avg_utilization": util
            })
            all_allocations[(K, s)] = df_alloc.copy()

    df_summary = pd.DataFrame(summary_rows)
    df_summary.to_csv(OUTPUT_DIR / "analysis1_summary_metrics_geojson.csv", index=False)

    print("Simulation summary (Strategies A, B, C):")
    display(df_summary)
    print("--- Analysis 1 Complete ---")
    return df_summary, all_allocations


def run_analysis_2_sensitivity(df_base, K_focus=100):
    """
    Runs the sensitivity analysis, focused only on K=100.
    """
    print(f"\n--- Running Analysis 2: Sensitivity (K={K_focus}) ---")
    freqs = [0.1, 0.2, 0.3, 0.4]
    services = [30, 45, 60, 90]
    strategies = ["A", "B", "C"]
    summary_records_sens = []

    for strat in strategies:
        df_alloc = allocate_chargers(df_base, K_focus, strat)
        for f in freqs:
            for s in services:
                sim = run_queue_simulation(df_alloc, freq_per_bev_week=f, mean_service_minutes=s, seed=123)

                valid_waits = [v["avg_wait_min"] for v in sim.values() if v["avg_wait_min"] is not None]
                avg_wait = np.mean(valid_waits) if valid_waits else np.nan # Use np.mean

                valid_util = [v["utilization"] for v in sim.values()]
                util = np.mean(valid_util) if valid_util else np.nan # Use np.mean

                summary_records_sens.append({
                    "K": K_focus, "Strategy": strat, "freq": f, "service_min": s,
                    "Avg_wait_min": avg_wait, "Avg_utilization": util
                })

    df_sensitivity = pd.DataFrame(summary_records_sens)
    df_sensitivity.to_csv(OUTPUT_DIR / f"analysis2_sensitivity_summary_K{K_focus}_geojson.csv", index=False)

    print(f"Sensitivity summary for K={K_focus} (sample):")
    display(df_sensitivity.round(3).head())
    print("--- Analysis 2 Complete ---")
    return df_sensitivity


def generate_k100_heatmaps(df_base, canon_coords, all_allocations):
    """
    Generates Folium heatmaps + new markers for K=100 allocations.
    """
    print("\n--- Running Analysis 4: Generating K=100 Heatmaps ---")
    K = 100
    strategies = ["A", "B", "C"]
    berlin_center_latlon = [52.5200, 13.4050] # lat, lon

    existing_locations = []
    for district in canon_coords:
        if district in POP_BY_DISTRICT: # Only include valid districts
            for lon, lat in canon_coords[district]:
                existing_locations.append([lat, lon]) # HeatMap expects [lat, lon]

    if not existing_locations:
        print("⚠️ WARNING: No existing charger locations found in GeoJSON data. Heatmaps will only show new locations.")

    for s in strategies:
        strat_name = {"A": "Max_Equity", "B": "Balanced", "C": "Max_Utility"}[s]

        if (K, s) not in all_allocations:
            print(f"Warning: Allocation for (K={K}, Strategy={s}) not found. Skipping heatmap.")
            continue

        df_alloc = all_allocations[(K, s)]

        new_locations_markers = []
        new_locations_heat = []

        for district, row in df_alloc.iterrows():
            new_chargers_count = int(row["new_alloc"])
            if new_chargers_count > 0:
                # Use centroids calculated during baseline prep
                lon = df_base.at[district, "centroid_x"]
                lat = df_base.at[district, "centroid_y"]

                new_locations_markers.append({
                    "location": [lat, lon],
                    "popup": f"New Allocation: {new_chargers_count}\nDistrict: {district}\nStrategy: {strat_name}"
                })
                for _ in range(new_chargers_count):
                    # Add jitter to heatmap points to avoid perfect overlap at centroid
                    new_locations_heat.append([lat + random.uniform(-0.001, 0.001),
                                               lon + random.uniform(-0.001, 0.001)])


        all_locations_heat = existing_locations + new_locations_heat

        if not all_locations_heat:
             print(f"⚠️ WARNING: No locations (existing or new) found for K={K}, Strat={s}. Skipping heatmap.")
             continue


        m = folium.Map(location=berlin_center_latlon, zoom_start=10, tiles="CartoDB positron")

        HeatMap(all_locations_heat, radius=10, blur=15).add_to(m)

        for marker_info in new_locations_markers:
            folium.Marker(
                location=marker_info["location"],
                popup=marker_info["popup"],
                icon=folium.Icon(color='red', icon='charging-station', prefix='fa')
            ).add_to(m)

        map_filename = OUTPUT_DIR / f"analysis4_heatmap_K{K}_Strat_{strat_name}_geojson.html"
        m.save(str(map_filename))
        print(f"✅ Heatmap for {strat_name} saved to: {map_filename}")

    print("--- Analysis 4 Complete ---")


def generate_nsga2_plot():
    """
    Loads the user-provided 'nsga2_all_k_results.csv' and replicates
    the 'nsga2_pareto_fronts_multi_k.jpg' plot.
    """
    print(f"\n--- Replicating NSGA-II Plot from {NSGA2_FILE_PATH} ---")
    if not os.path.exists(NSGA2_FILE_PATH):
        print(f"❌ Error: {NSGA2_FILE_PATH} not found. Skipping NSGA-II plot.")
        return

    try:
        df_nsga = pd.read_csv(NSGA2_FILE_PATH)
        df_nsga = df_nsga.dropna(subset=['Gini_post', 'Weighted_Coverage_Score'])

        plt.figure(figsize=(10, 7))
        k_values = sorted(df_nsga['K'].unique())
        colors = plt.cm.viridis(np.linspace(0, 1, len(k_values)))

        for k, color in zip(k_values, colors):
            k_data = df_nsga[df_nsga['K'] == k]
            plt.scatter(
                k_data['Gini_post'],
                # Assuming the CSV stores Avg_Wait_Min here as per generator script
                k_data['Weighted_Coverage_Score'],
                color=color, label=f'K={k}', alpha=0.7, s=30
            )

        plt.xlabel("Gini Coefficient (Lower is Better)")
        # Correct label based on how the generator script saves the data
        plt.ylabel("Average Wait Time (min) (Higher is Worse)")
        plt.title("NSGA-II Pareto Fronts (from nsga2_all_k_results.csv)")
        plt.legend(title="K Values")
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.gca().invert_xaxis() # Gini: Lower is better
        # Do NOT invert Y axis: Lower wait time is better

        plot_filename = OUTPUT_DIR / "nsga2_pareto_fronts_replicated.png"
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        print(f"✅ NSGA-II plot saved to: {plot_filename}")
        # plt.show() # Prevent showing plot in this environment
        plt.close() # Close plot figure

    except Exception as e:
        print(f"❌ Error generating NSGA-II plot: {e}")


def generate_avg_times_plot(df_summary):
    """
    Replicates the 'Average times.jpeg' plot using simulation data for K=100.
    """
    print("\n--- Replicating 'Average Times' Plot (K=100, Strategies A,B,C) ---")
    try:
        # Ensure the summary dataframe has the correct columns and K=100 data
        if df_summary is None or 'K' not in df_summary.columns or 'Avg_wait_min' not in df_summary.columns:
             print("❌ Error: Simulation summary data is missing or incomplete. Cannot generate plot.")
             return

        df_k100 = df_summary[df_summary['K'] == 100].copy()

        if df_k100.empty or df_k100['Avg_wait_min'].isnull().all():
            print("⚠️ Warning: No valid K=100 simulation data found. Skipping 'Average Times' plot.")
            return

        df_k100['Strategy_Name'] = df_k100['Strategy'].map({
            'A': 'Max Equity', 'B': 'Balanced', 'C': 'Max Utility'
        })

        plt.figure(figsize=(8, 5))
        barplot = sns.barplot(
            x='Strategy_Name',
            y='Avg_wait_min',
            data=df_k100,
            palette='muted',
            order=['Max Equity', 'Balanced', 'Max Utility']
        )

        plt.title("Average Wait Times for K=100 Allocation (Strategies A, B, C)")
        plt.xlabel("Allocation Strategy")
        plt.ylabel("Average Wait Time (minutes)")

        # Add labels, handle potential NaN values gracefully
        for p in barplot.patches:
             height = p.get_height()
             if pd.notna(height):
                 barplot.annotate(
                     f"{height:.2f} min",
                     (p.get_x() + p.get_width() / 2., height),
                     ha='center', va='center', xytext=(0, 9), textcoords='offset points'
                 )

        plot_filename = OUTPUT_DIR / "average_wait_times_replicated_K100_geojson.png"
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        print(f"✅ Average times bar chart saved to: {plot_filename}")
        # plt.show() # Prevent showing plot
        plt.close() # Close plot figure

    except Exception as e:
        print(f"❌ Error generating average times plot: {e}")


# --- 3. Main Execution ---

def main():
    """
    Main function to run the complete analysis pipeline using GeoJSON.
    """
    OUTPUT_DIR.mkdir(exist_ok=True)

    # --- Step 1: Check for Required Files ---
    if not os.path.exists(CHARGERS_GEOJSON_PATH):
        print("="*50)
        print(f"❌ CRITICAL ERROR: File not found: {CHARGERS_GEOJSON_PATH}")
        print("Please ensure 'chargers_locations.geojson' is uploaded.")
        print("="*50)
        return

    if not os.path.exists(NSGA2_FILE_PATH):
        print("="*50)
        print(f"⚠️ WARNING: File not found: {NSGA2_FILE_PATH}")
        print("The NSGA-II Pareto front plot will not be generated.")
        print("="*50)

    # --- Step 2: Prepare Baseline Data from GeoJSON ---
    df_base, canon_coords = prepare_baseline_data_from_geojson(CHARGERS_GEOJSON_PATH)
    if df_base is None:
        print("Halting script due to data preparation failure.")
        return

    # --- Step 3: Run Analyses (A, B, C) ---
    df_summary, all_allocations = run_analysis_1_district_sim(df_base)
    run_analysis_2_sensitivity(df_base, K_focus=100)

    # --- Step 4: Generate Required Outputs ---
    generate_k100_heatmaps(df_base, canon_coords, all_allocations)
    generate_avg_times_plot(df_summary)
    generate_nsga2_plot() # This reads the static CSV

    print("\n==================================")
    print("All analyses complete.")
    print(f"All outputs saved in: {OUTPUT_DIR.resolve()}")
    print("==================================")

if __name__ == "__main__":
    # Set plot style
    sns.set_theme(style="whitegrid")

    # Run the main pipeline
    main()

Preparing baseline data from chargers_locations.geojson...
Found 3237 chargers commissioned by 2024-12-31.
✅ Baseline data prepared successfully from GeoJSON.
Total existing chargers found: 5063

--- Running Analysis 1: District-Level Simulation (A, B, C) ---
Simulation summary (Strategies A, B, C):


,K,Strategy,Gini_post,Coverage_pct,Avg_wait_min,Pct_arrivals_waited,Avg_utilization
0,100,A,0.149814,100.0,0.0,0.0,0.009443
1,100,B,0.149814,100.0,0.0,0.0,0.009443
2,100,C,0.150407,100.0,0.0,0.0,0.009449
3,150,A,0.148633,100.0,0.0,0.0,0.009339
4,150,B,0.148633,100.0,0.0,0.0,0.009339
5,150,C,0.148755,100.0,0.0,0.0,0.009342
6,500,A,0.141281,100.0,0.0,0.0,0.008676
7,500,B,0.141281,100.0,0.0,0.0,0.008676
8,500,C,0.141391,100.0,0.0,0.0,0.008678
9,1000,A,0.132052,100.0,0.0,0.0,0.007881


--- Analysis 1 Complete ---

--- Running Analysis 2: Sensitivity (K=100) ---
Sensitivity summary for K=100 (sample):


,K,Strategy,freq,service_min,Avg_wait_min,Avg_utilization
0,100,A,0.1,30,0.0,0.003
1,100,A,0.1,45,0.0,0.005
2,100,A,0.1,60,0.0,0.006
3,100,A,0.1,90,0.0,0.010
4,100,A,0.2,30,0.0,0.006


--- Analysis 2 Complete ---

--- Running Analysis 4: Generating K=100 Heatmaps ---
✅ Heatmap for Max_Equity saved to: simulation_outputs_final_geojson/analysis4_heatmap_K100_Strat_Max_Equity_geojson.html
✅ Heatmap for Balanced saved to: simulation_outputs_final_geojson/analysis4_heatmap_K100_Strat_Balanced_geojson.html
✅ Heatmap for Max_Utility saved to: simulation_outputs_final_geojson/analysis4_heatmap_K100_Strat_Max_Utility_geojson.html
--- Analysis 4 Complete ---

--- Replicating 'Average Times' Plot (K=100, Strategies A,B,C) ---


/tmp/ipython-input-3133234274.py:488: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  barplot = sns.barplot(


✅ Average times bar chart saved to: simulation_outputs_final_geojson/average_wait_times_replicated_K100_geojson.png

--- Replicating NSGA-II Plot from nsga2_all_k_results.csv ---
✅ NSGA-II plot saved to: simulation_outputs_final_geojson/nsga2_pareto_fronts_replicated.png

All analyses complete.
All outputs saved in: /content/simulation_outputs_final_geojson
